## Start

In [10]:
import importlib
from pathlib import Path
from pprint import pprint
import json
import re
import pandas as pd
import numpy as np
from datetime import datetime

from langchain_core.messages import HumanMessage, AIMessage, ToolMessage


In [4]:
def print_resp(resp):
    step_num = 1
    for message in resp["messages"]:
        if isinstance(message, HumanMessage):
            print(f"Step {step_num} - inputs:")
            print(f"   {message.content[:200]}..." if len(message.content) > 200 else f"   {message.content}")
            print()
            step_num += 1

        elif isinstance(message, AIMessage):
            if hasattr(message, 'tool_calls') and message.tool_calls:
                # Agent 决定调用工具
                print(f"Step {step_num} - Agent decide tools used:")
                for tool_call in message.tool_calls:
                    print(f"   Tool name: {tool_call['name']}")
                    print(f"   Tool parameters: {tool_call['args']}")
                print()
                step_num += 1
            elif message.content:
                print(f"Step {step_num} - Agent outputs:")
                print(f"   {message.content}")
                print()
                step_num += 1

        elif isinstance(message, ToolMessage):
            print(f"Step {step_num} - outputs:")
            print(f"   Tool name: {message.name}")
            # result_preview = message.content[:300] + "..." if len(message.content) > 300 else message.content
            result_preview = message.content
            print(f"   Outputs: {result_preview}")
            print()
            step_num += 1

    print(f"\n{'='*80}")
    print(f"Final outputs:")
    print(f"{'='*80}\n")
    print(resp["messages"][-1].content)


In [5]:
from bsm_multi_agents.config import llm_config
importlib.reload(llm_config)
from bsm_multi_agents.config.llm_config import get_llm

from bsm_multi_agents import tools
importlib.reload(tools)
from bsm_multi_agents.tools import get_tools_for_role

from bsm_multi_agents.agents import agent_factory
importlib.reload(agent_factory)
from bsm_multi_agents.agents.agent_factory import built_graph_agent,built_graph_agent_by_role
from bsm_multi_agents.agents.utils import merge_state_update_from_tool_messages

from bsm_multi_agents.prompts import loader
importlib.reload(loader)
from bsm_multi_agents.prompts.loader import load_prompt



In [6]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

## Data Loader

In [53]:
from bsm_multi_agents.agents import data_loader_agent
importlib.reload(data_loader_agent)
from bsm_multi_agents.agents.data_loader_agent import data_loader_node

In [54]:
csv_path = str(Path.cwd().parents[1] / "data" / "input" / "dummy_options.csv")
state = WorkflowState(csv_file_path=csv_path)

In [55]:
out = data_loader_node(state)

In [39]:
print_resp(out)

Step 1 - inputs:
   "Load the option data from the CSV file at: /Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv"

"Use the csv_loader tool to read the CSV file. Return the data in JSON format."

Step 2 - Agent decide tools used:
   Tool name: csv_loader
   Tool parameters: {'filepath': '/Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv'}

Step 3 - outputs:
   Tool name: csv_loader
   Outputs: {"state_update": {"csv_data": [{"date": "2025-09-01", "S": 100, "K": 105, "T": 1.0, "r": 0.05, "sigma": 0.2, "option_type": "call"}, {"date": "2025-09-02", "S": 102, "K": 106, "T": 0.9, "r": 0.045, "sigma": 0.19, "option_type": "put"}, {"date": "2025-09-03", "S": 98, "K": 104, "T": 0.8, "r": 0.048, "sigma": 0.21, "option_type": "call"}, {"date": "2025-09-04", "S": 101, "K": 107, "T": 0.7, "r": 0.047, "sigma": 0.18, "option_type": "call"}, {"date": "2025-09-05", "S": 99, "K": 103, "T": 0.6, "r": 0.046, "sigma": 0.22, "option_type": "put"}, {"date": "2025-09-06", "S

In [46]:
new_state = {**state, **out}
new_state

{'csv_file_path': '/Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv',
 'messages': [HumanMessage(content='"Load the option data from the CSV file at: /Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv"\n\n"Use the csv_loader tool to read the CSV file. Return the data in JSON format."', additional_kwargs={}, response_metadata={}, id='2c178c97-402d-4bd9-8527-ace18b72965f'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-11-07T16:36:34.799868Z', 'done': True, 'done_reason': 'stop', 'total_duration': 516747625, 'load_duration': 63325917, 'prompt_eval_count': 266, 'prompt_eval_duration': 58645791, 'eval_count': 35, 'eval_duration': 381970707, 'model_name': 'qwen2.5:7b', 'model_provider': 'ollama'}, id='lc_run--ebc3aeae-6191-4671-8a46-9e0fd5826339-0', tool_calls=[{'name': 'csv_loader', 'args': {'filepath': '/Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv'}, 'id': '95dd434a-6c57-4160

### Check data_loader_node

In [56]:
from bsm_multi_agents.agents.utils import merge_state_update_from_tool_messages

In [57]:
from bsm_multi_agents.agents.agent_factory import built_graph_agent_by_role
agent_role = 'data_loader'
agent = built_graph_agent_by_role(agent_role)

csv_path = state.get("csv_file_path")
if not csv_path:
    csv_path = str(Path.cwd().parents[1] / "data" / "input" / "dummy_options.csv")

prompt_path = Path.cwd().parents[1] / "src" / "bsm_multi_agents" / "prompts" / "data_loader_prompts.txt"
prompt = load_prompt(prompt_path).format(csv_path=str(csv_path))
msg = HumanMessage(content=prompt)

result = agent.invoke(
    {"messages": [msg]},
    config={
        "recursion_limit": 10,
        "configurable": {"thread_id": "run-1"}
    }
)
merged_messages = list(state.get("messages", []))
if isinstance(result, dict) and "messages" in result:
    merged_messages.extend(result["messages"])
out = {"messages": merged_messages}




## Calculator

In [13]:
from bsm_multi_agents.agents import data_loader_agent
importlib.reload(data_loader_agent)
from bsm_multi_agents.agents.data_loader_agent import data_loader_node

In [14]:
csv_path = str(Path.cwd().parents[1] / "data" / "input" / "dummy_options.csv")
state = WorkflowState(csv_file_path=csv_path)
out = data_loader_node(state)
state = {**state, **out}

In [15]:
from bsm_multi_agents.agents import calculator_agent
importlib.reload(calculator_agent)
from bsm_multi_agents.agents.calculator_agent import calculator_node

In [16]:
out = calculator_node(state)

In [17]:
out

{'messages': [HumanMessage(content='"Load the option data from the CSV file at: /Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv"\n\n"Use the csv_loader tool to read the CSV file. Return the data in JSON format."', additional_kwargs={}, response_metadata={}, id='0bab76df-fd96-43e3-a2da-3462bb47f6b7'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-11-07T19:41:06.851596Z', 'done': True, 'done_reason': 'stop', 'total_duration': 842614666, 'load_duration': 77347375, 'prompt_eval_count': 266, 'prompt_eval_duration': 368084667, 'eval_count': 35, 'eval_duration': 385281379, 'model_name': 'qwen2.5:7b', 'model_provider': 'ollama'}, id='lc_run--ccbb363a-129a-49b7-9c05-b47cde02aa10-0', tool_calls=[{'name': 'csv_loader', 'args': {'filepath': '/Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv'}, 'id': '51980756-5f0f-442c-a9da-8b238577bbc0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 266, 'outp

In [73]:
print_resp(out)

Step 1 - inputs:
   "Load the option data from the CSV file at: /Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv"

"Use the csv_loader tool to read the CSV file. Return the data in JSON format."

Step 2 - Agent decide tools used:
   Tool name: csv_loader
   Tool parameters: {'filepath': '/Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv'}

Step 3 - outputs:
   Tool name: csv_loader
   Outputs: {"state_update": {"csv_data": [{"date": "2025-09-01", "S": 100, "K": 105, "T": 1.0, "r": 0.05, "sigma": 0.2, "option_type": "call"}, {"date": "2025-09-02", "S": 102, "K": 106, "T": 0.9, "r": 0.045, "sigma": 0.19, "option_type": "put"}, {"date": "2025-09-03", "S": 98, "K": 104, "T": 0.8, "r": 0.048, "sigma": 0.21, "option_type": "call"}, {"date": "2025-09-04", "S": 101, "K": 107, "T": 0.7, "r": 0.047, "sigma": 0.18, "option_type": "call"}, {"date": "2025-09-05", "S": 99, "K": 103, "T": 0.6, "r": 0.046, "sigma": 0.22, "option_type": "put"}, {"date": "2025-09-06", "S

In [26]:
new_state = {**state, **out}
new_state

{'csv_file_path': '/Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv',
 'messages': [HumanMessage(content='"Load the option data from the CSV file at: /Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv"\n\n"Use the csv_loader tool to read the CSV file. Return the data in JSON format."', additional_kwargs={}, response_metadata={}, id='92c5edb5-d38d-4628-89cd-cab622782223'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-11-07T17:15:44.790134Z', 'done': True, 'done_reason': 'stop', 'total_duration': 516927833, 'load_duration': 63178166, 'prompt_eval_count': 266, 'prompt_eval_duration': 62115541, 'eval_count': 35, 'eval_duration': 379619837, 'model_name': 'qwen2.5:7b', 'model_provider': 'ollama'}, id='lc_run--158bccfd-cf2f-4887-9586-84e5726fe4ea-0', tool_calls=[{'name': 'csv_loader', 'args': {'filepath': '/Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv'}, 'id': 'cf9e41ee-bdae-4290

### Check calculator_node

In [5]:
from bsm_multi_agents.agents.agent_factory import built_graph_agent_by_role

from bsm_multi_agents.agents import data_loader_agent
importlib.reload(data_loader_agent)
from bsm_multi_agents.agents.data_loader_agent import data_loader_node

In [6]:
csv_path = str(Path.cwd().parents[1] / "data" / "input" / "dummy_options.csv")
state = WorkflowState(csv_file_path=csv_path)
out = data_loader_node(state)
state = {**state, **out}

In [7]:
agent_role = "calculator"
default_system = """
You are a quantitative calculator agent.
"""
agent = built_graph_agent_by_role(agent_role,default_system=default_system)
prompt_path = Path.cwd().parents[1] / "src" / "bsm_multi_agents" / "prompts" / "calculator_prompts.txt"
csv_json = json.dumps(state["csv_data"], ensure_ascii=False)
user_prompt = load_prompt(prompt_path).format(csv_data_json=csv_json)
result = agent.invoke(
    {"messages": [HumanMessage(content=user_prompt)]},
    config={"recursion_limit": 10, "configurable": {"thread_id": state.get("thread_id","run-1")}}
)

In [8]:
print_resp(result)

Step 1 - inputs:
   "Calculate Black-Scholes-Merton option prices and greeks.
- Use the `batch_bsm_calculator` tool to calculate prices.
- Use the `batch_greeks_calculator` tool to calculate greeks.
- The input data is p...

Step 2 - Agent decide tools used:
   Tool name: batch_bsm_calculator
   Tool parameters: {'csv_data': [{'K': 105, 'S': 100, 'T': 1, 'date': '2025-09-01', 'option_type': 'call', 'r': 0.05, 'sigma': 0.2}, {'K': 106, 'S': 102, 'T': 0.9, 'date': '2025-09-02', 'option_type': 'put', 'r': 0.045, 'sigma': 0.19}, {'K': 104, 'S': 98, 'T': 0.8, 'date': '2025-09-03', 'option_type': 'call', 'r': 0.048, 'sigma': 0.21}, {'K': 107, 'S': 101, 'T': 0.7, 'date': '2025-09-04', 'option_type': 'call', 'r': 0.047, 'sigma': 0.18}, {'K': 103, 'S': 99, 'T': 0.6, 'date': '2025-09-05', 'option_type': 'put', 'r': 0.046, 'sigma': 0.22}, {'K': 108, 'S': 103, 'T': 0.5, 'date': '2025-09-06', 'option_type': 'put', 'r': 0.049, 'sigma': 0.2}, {'K': 102, 'S': 97, 'T': 0.4, 'date': '2025-09-07', 'optio

In [9]:
from bsm_multi_agents.agents.utils import merge_state_update_from_tool_messages
merged_messages = list(state.get("messages", []))
if isinstance(result, dict) and "messages" in result:
    merged_messages.extend(result["messages"])
out = {"messages": merged_messages}

merge_state_update_from_tool_messages(
    result,
    out,
    tool_names=("batch_bsm_calculator", "batch_greeks_calculator"),
)

In [10]:
out

{'messages': [HumanMessage(content='"Load the option data from the CSV file at: /Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv"\n\n"Use the csv_loader tool to read the CSV file. Return the data in JSON format."', additional_kwargs={}, response_metadata={}, id='25a6ed7d-74c6-4305-98af-58ff2d80c8fc'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-11-07T19:59:43.763788Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1281964333, 'load_duration': 562296708, 'prompt_eval_count': 266, 'prompt_eval_duration': 319285959, 'eval_count': 35, 'eval_duration': 387512916, 'model_name': 'qwen2.5:7b', 'model_provider': 'ollama'}, id='lc_run--241e2e69-50e0-463c-9d4b-38c8374327d1-0', tool_calls=[{'name': 'csv_loader', 'args': {'filepath': '/Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv'}, 'id': '93ca72c4-4ebb-4636-8cb9-233aadc46116', 'type': 'tool_call'}], usage_metadata={'input_tokens': 266, 'ou

In [ ]:
from bsm_multi_agents.agents.utils import merge_state_update_from_tool_messages
merged_messages = list(state.get("messages", []))
if isinstance(result, dict) and "messages" in result:
    merged_messages.extend(result["messages"])
out = {"messages": merged_messages}
merge_state_update_from_tool_messages(result, out, tool_names=(
        "batch_greeks_calculator",
        "batch_greeks_calculator"
    ))

In [64]:
out

{'messages': [HumanMessage(content='"Load the option data from the CSV file at: /Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv"\n\n"Use the csv_loader tool to read the CSV file. Return the data in JSON format."', additional_kwargs={}, response_metadata={}, id='5445538a-60fb-4bbd-903b-836f2617640a'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-11-07T19:04:48.618874Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1554259959, 'load_duration': 809256084, 'prompt_eval_count': 266, 'prompt_eval_duration': 341360041, 'eval_count': 35, 'eval_duration': 384511714, 'model_name': 'qwen2.5:7b', 'model_provider': 'ollama'}, id='lc_run--dd783d62-d552-4cce-8b0d-0539256c3b8f-0', tool_calls=[{'name': 'csv_loader', 'args': {'filepath': '/Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv'}, 'id': '938f79bb-f39d-462e-9d1e-7de6597ec4f7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 266, 'ou

## Validator

In [ ]:
from bsm_multi_agents.agents import data_loader_agent
importlib.reload(data_loader_agent)
from bsm_multi_agents.agents.data_loader_agent import data_loader_node

In [ ]:
csv_path = str(Path.cwd().parents[1] / "data" / "input" / "dummy_options.csv")
state = WorkflowState(csv_file_path=csv_path)
out = data_loader_node(state)
state = {**state, **out}

In [ ]:
from bsm_multi_agents.agents import calculator_agent
importlib.reload(calculator_agent)
from bsm_multi_agents.agents.calculator_agent import calculator_node

In [ ]:
out = calculator_node(state)
state = {**state, **out}

### Check validator_node

In [ ]:
from bsm_multi_agents.agents import data_loader_agent
importlib.reload(data_loader_agent)
from bsm_multi_agents.agents.data_loader_agent import data_loader_node

In [6]:
csv_path = str(Path.cwd().parents[1] / "data" / "input" / "dummy_options.csv")
state = WorkflowState(csv_file_path=csv_path)
out = data_loader_node(state)
state = {**state, **out}

In [7]:
from bsm_multi_agents.agents import calculator_agent
importlib.reload(calculator_agent)
from bsm_multi_agents.agents.calculator_agent import calculator_node

In [8]:
out = calculator_node(state)
state = {**state, **out}

In [10]:
"greeks_results" not in state or not state["greeks_results"]

False

In [11]:
agent_role = "validator"
default_system = """
You are a quantitative validator agent.
"""
agent = built_graph_agent_by_role(agent_role,default_system=default_system)

In [21]:
prompt_path = Path.cwd().parents[1] / "src" / "bsm_multi_agents" / "prompts" / "validator_prompts.txt"
greeks_results_json = json.dumps(state["greeks_results"], ensure_ascii=False)
user_prompt = load_prompt(prompt_path).format(greeks_results=greeks_results_json)

In [22]:
result = agent.invoke(
    {"messages": [HumanMessage(content=user_prompt)]},
    config={"recursion_limit": 10, "configurable": {"thread_id": state.get("thread_id","run-1")}}
)

In [27]:
merged_messages = list(state.get("messages", []))
if isinstance(result, dict) and "messages" in result:
    merged_messages.extend(result["messages"])
out = {"messages": merged_messages}
merge_state_update_from_tool_messages(
    result,
    out,
    tool_names=("batch_greeks_validator",),
)
out['validate_results']

[{'K': 105,
  'S': 100,
  'T': 1.0,
  'date': '2025-09-01',
  'delta': 0.5422283336,
  'gamma': 0.0198352619,
  'option_type': 'call',
  'price': 8.0213522351,
  'r': 0.05,
  'rho': 46.2014811233,
  'sigma': 0.2,
  'theta': -6.277126437,
  'vega': 39.6705238084,
  'validations_result': 'passed',
  'validations_details': []},
 {'K': 106,
  'S': 102,
  'T': 0.9,
  'date': '2025-09-02',
  'delta': -0.4596134147,
  'gamma': 0.0215874826,
  'option_type': 'put',
  'price': 7.2142383713,
  'r': 0.045,
  'rho': -48.685326005,
  'sigma': 0.19,
  'theta': -1.6196945478,
  'vega': 38.4059448764,
  'validations_result': 'passed',
  'validations_details': []},
 {'K': 104,
  'S': 98,
  'T': 0.8,
  'date': '2025-09-03',
  'delta': 0.4928141481,
  'gamma': 0.0216695176,
  'option_type': 'call',
  'price': 6.415749359,
  'r': 0.048,
  'rho': 33.5040297211,
  'sigma': 0.21,
  'theta': -6.599156514,
  'vega': 34.963159853,
  'validations_result': 'passed',
  'validations_details': []},
 {'K': 107,
  'S'

## Summary Generator

In [14]:
from bsm_multi_agents.agents import data_loader_agent
importlib.reload(data_loader_agent)
from bsm_multi_agents.agents.data_loader_agent import data_loader_node


from bsm_multi_agents.agents import calculator_agent
importlib.reload(calculator_agent)
from bsm_multi_agents.agents.calculator_agent import calculator_node


from bsm_multi_agents.agents import validator_agent
importlib.reload(validator_agent)
from bsm_multi_agents.agents.validator_agent import validator_node

from bsm_multi_agents.agents import summary_generator
importlib.reload(summary_generator)
from bsm_multi_agents.agents.summary_generator import summary_generator_node

In [15]:
csv_path = str(Path.cwd().parents[1] / "data" / "input" / "dummy_options.csv")
state = WorkflowState(csv_file_path=csv_path)
out = data_loader_node(state)
state = {**state, **out}

out = calculator_node(state)
state = {**state, **out}

out = validator_node(state)
state = {**state, **out}

out = summary_generator_node(state)
state = {**state, **out}

In [16]:
state

{'csv_file_path': '/Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv',
 'messages': [HumanMessage(content='"Load the option data from the CSV file at: /Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv"\n\n"Use the csv_loader tool to read the CSV file. Return the data in JSON format."', additional_kwargs={}, response_metadata={}, id='c184d4fa-c4b8-462f-bed5-461a824cb480'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-11-17T02:39:18.811638Z', 'done': True, 'done_reason': 'stop', 'total_duration': 852378875, 'load_duration': 80888792, 'prompt_eval_count': 266, 'prompt_eval_duration': 376037667, 'eval_count': 35, 'eval_duration': 383382167, 'model_name': 'qwen2.5:7b', 'model_provider': 'ollama'}, id='lc_run--35eb1d5c-8867-4b29-9499-757d24833b31-0', tool_calls=[{'name': 'csv_loader', 'args': {'filepath': '/Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv'}, 'id': '777545d9-ed8f-4be

In [17]:
print_resp(out)

Step 1 - inputs:
   "Load the option data from the CSV file at: /Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv"

"Use the csv_loader tool to read the CSV file. Return the data in JSON format."

Step 2 - Agent decide tools used:
   Tool name: csv_loader
   Tool parameters: {'filepath': '/Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv'}

Step 3 - outputs:
   Tool name: csv_loader
   Outputs: {"state_update": {"csv_data": [{"date": "2025-09-01", "S": 100, "K": 105, "T": 1.0, "r": 0.05, "sigma": 0.2, "option_type": "call"}, {"date": "2025-09-02", "S": 102, "K": 106, "T": 0.9, "r": 0.045, "sigma": 0.19, "option_type": "put"}, {"date": "2025-09-03", "S": 98, "K": 104, "T": 0.8, "r": 0.048, "sigma": 0.21, "option_type": "call"}, {"date": "2025-09-04", "S": 101, "K": 107, "T": 0.7, "r": 0.047, "sigma": 0.18, "option_type": "call"}, {"date": "2025-09-05", "S": 99, "K": 103, "T": 0.6, "r": 0.046, "sigma": 0.22, "option_type": "put"}, {"date": "2025-09-06", "S

### Check summary_generator_node

In [5]:
from bsm_multi_agents.agents import data_loader_agent
importlib.reload(data_loader_agent)
from bsm_multi_agents.agents.data_loader_agent import data_loader_node


from bsm_multi_agents.agents import calculator_agent
importlib.reload(calculator_agent)
from bsm_multi_agents.agents.calculator_agent import calculator_node


from bsm_multi_agents.agents import validator_agent
importlib.reload(validator_agent)
from bsm_multi_agents.agents.validator_agent import validator_node

In [6]:
csv_path = str(Path.cwd().parents[1] / "data" / "input" / "dummy_options.csv")
state = WorkflowState(csv_file_path=csv_path)
out = data_loader_node(state)
state = {**state, **out}

In [7]:
out = calculator_node(state)
state = {**state, **out}

In [8]:
out = validator_node(state)
state = {**state, **out}

In [9]:
"validate_results" not in state or not state["validate_results"]

False

In [10]:
agent_role = "summary_generator"
default_system = """
You are a reporting agent specialized in generating summary reports.
"""
agent = built_graph_agent_by_role(agent_role, default_system=default_system)

In [11]:
validate_results_str = json.dumps(state["validate_results"], ensure_ascii=False)
bsm_results_str = json.dumps(state.get("bsm_results", []), ensure_ascii=False)
greeks_results_str = json.dumps(state.get("greeks_results", []), ensure_ascii=False)

template_path = Path.cwd().parents[1] / "src" / "bsm_multi_agents" / "templates" / "summary_template.md"

prompt_path = Path.cwd().parents[1] / "src" / "bsm_multi_agents" / "prompts" / "summary_generator_prompts.txt"
user_prompt = load_prompt(prompt_path).format(
    validate_results=validate_results_str,
    bsm_results=bsm_results_str,
    greeks_results=greeks_results_str,
    template_path=template_path
)
user_prompt

'Call the tool `generate_summary` with EXACTLY these arguments (do NOT wrap them in a JSON string):\n\nvalidate_results = [{"K": 105, "S": 100, "T": 1.0, "date": "2025-09-01", "delta": 0.5422283336, "gamma": 0.0198352619, "option_type": "call", "price": 8.0213522351, "r": 0.05, "rho": 46.2014811233, "sigma": 0.2, "theta": -6.277126437, "vega": 39.6705238084, "validations_result": "passed", "validations_details": []}, {"K": 106, "S": 102, "T": 0.9, "date": "2025-09-02", "delta": -0.4596134147, "gamma": 0.0215874826, "option_type": "put", "price": 7.2142383713, "r": 0.045, "rho": -48.685326005, "sigma": 0.19, "theta": -1.6196945478, "vega": 38.4059448764, "validations_result": "passed", "validations_details": []}, {"K": 104, "S": 98, "T": 0.8, "date": "2025-09-03", "delta": 0.4928141481, "gamma": 0.0216695176, "option_type": "call", "price": 6.415749359, "r": 0.048, "rho": 33.5040297211, "sigma": 0.21, "theta": -6.599156514, "vega": 34.963159853, "validations_result": "passed", "validati

In [12]:
result = agent.invoke(
    {"messages": [HumanMessage(content=user_prompt)]},
    config={"recursion_limit": 10, "configurable": {"thread_id": state.get("thread_id","run-1")}}
)

In [13]:
result

{'messages': [HumanMessage(content='Call the tool `generate_summary` with EXACTLY these arguments (do NOT wrap them in a JSON string):\n\nvalidate_results = [{"K": 105, "S": 100, "T": 1.0, "date": "2025-09-01", "delta": 0.5422283336, "gamma": 0.0198352619, "option_type": "call", "price": 8.0213522351, "r": 0.05, "rho": 46.2014811233, "sigma": 0.2, "theta": -6.277126437, "vega": 39.6705238084, "validations_result": "passed", "validations_details": []}, {"K": 106, "S": 102, "T": 0.9, "date": "2025-09-02", "delta": -0.4596134147, "gamma": 0.0215874826, "option_type": "put", "price": 7.2142383713, "r": 0.045, "rho": -48.685326005, "sigma": 0.19, "theta": -1.6196945478, "vega": 38.4059448764, "validations_result": "passed", "validations_details": []}, {"K": 104, "S": 98, "T": 0.8, "date": "2025-09-03", "delta": 0.4928141481, "gamma": 0.0216695176, "option_type": "call", "price": 6.415749359, "r": 0.048, "rho": 33.5040297211, "sigma": 0.21, "theta": -6.599156514, "vega": 34.963159853, "valid

In [14]:
merged_messages = list(state.get("messages", []))
if isinstance(result, dict) and "messages" in result:
    merged_messages.extend(result["messages"])
out = {"messages": merged_messages}

In [15]:
merge_state_update_from_tool_messages(
    result,
    out,
    tool_names=("generate_summary",),
)

In [16]:
out

{'messages': [HumanMessage(content='"Load the option data from the CSV file at: /Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv"\n\n"Use the csv_loader tool to read the CSV file. Return the data in JSON format."', additional_kwargs={}, response_metadata={}, id='d6061fda-d914-4ba6-af5f-eb887c85362d'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-11-10T02:40:57.005363Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1249065625, 'load_duration': 528412333, 'prompt_eval_count': 266, 'prompt_eval_duration': 324612333, 'eval_count': 35, 'eval_duration': 382634497, 'model_name': 'qwen2.5:7b', 'model_provider': 'ollama'}, id='lc_run--bb4ced4e-a5be-4613-b3d6-ede94f5ff65d-0', tool_calls=[{'name': 'csv_loader', 'args': {'filepath': '/Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv'}, 'id': '29ee46c3-747e-43d4-8147-2521a107d8ad', 'type': 'tool_call'}], usage_metadata={'input_tokens': 266, 'ou

In [17]:
print_resp(out)

Step 1 - inputs:
   "Load the option data from the CSV file at: /Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv"

"Use the csv_loader tool to read the CSV file. Return the data in JSON format."

Step 2 - Agent decide tools used:
   Tool name: csv_loader
   Tool parameters: {'filepath': '/Users/yifanli/Github/langgraph_test/data/input/dummy_options.csv'}

Step 3 - outputs:
   Tool name: csv_loader
   Outputs: {"state_update": {"csv_data": [{"date": "2025-09-01", "S": 100, "K": 105, "T": 1.0, "r": 0.05, "sigma": 0.2, "option_type": "call"}, {"date": "2025-09-02", "S": 102, "K": 106, "T": 0.9, "r": 0.045, "sigma": 0.19, "option_type": "put"}, {"date": "2025-09-03", "S": 98, "K": 104, "T": 0.8, "r": 0.048, "sigma": 0.21, "option_type": "call"}, {"date": "2025-09-04", "S": 101, "K": 107, "T": 0.7, "r": 0.047, "sigma": 0.18, "option_type": "call"}, {"date": "2025-09-05", "S": 99, "K": 103, "T": 0.6, "r": 0.046, "sigma": 0.22, "option_type": "put"}, {"date": "2025-09-06", "S

### Check generate summary tool

In [58]:
from typing import Optional, Union, Dict, Any, List
from bsm_multi_agents.tools.utils import load_json_as_df 
def _load_template_text(template_path: Optional[str]) -> Optional[str]:
    """
    If template_path provided and exists, use it.
    Else try default: src/templates/summary_template.md
    """
    path = None
    if template_path:
        p = Path(template_path)
        path = p if p.exists() else None
    if path is None:
        default = Path(__file__).resolve().parents[1] / "templates" / "summary_template.md"
        path = default if default.exists() else None
    return path.read_text(encoding="utf-8") if path else None

def _generate_greeks_summary(vr_df: pd.DataFrame) -> str:
    """生成 Greeks 摘要"""
    lines = []
    greek_cols = ['delta', 'gamma', 'vega', 'theta', 'rho']

    for col in greek_cols:
        if col in vr_df.columns:
            try:
                col_numeric = pd.to_numeric(vr_df[col], errors='coerce')
                lines.append(f"- **{col.capitalize()}:** Avg = {col_numeric.mean():.6f}, Range = [{col_numeric.min():.6f}, {col_numeric.max():.6f}]")
            except Exception:
                pass

    return "\n".join(lines) if lines else "- Greeks data not available"


def _generate_greek_analysis(vr_df: pd.DataFrame, col_name: str, display_name: str, expected_range: str) -> str:
    """生成单个 Greek 的详细分析"""
    if col_name not in vr_df.columns:
        return f"- **{display_name}** data not available in validation results"

    try:
        col_numeric = pd.to_numeric(vr_df[col_name], errors='coerce')

        analysis = f"- **Portfolio {display_name}:** {col_numeric.sum():.6f}\n"
        analysis += f"- **Average {display_name}:** {col_numeric.mean():.6f}\n"
        analysis += f"- **{display_name} Range:** [{col_numeric.min():.6f}, {col_numeric.max():.6f}]\n"
        analysis += f"- **Expected Range:** {expected_range}\n"
        analysis += f"- **Standard Deviation:** {col_numeric.std():.6f}"

        return analysis
    except Exception:
        return f"- Unable to compute {display_name} statistics"


def _generate_recommendations(fail_cnt: Optional[int], total: int, vr_df: pd.DataFrame) -> str:
    """生成建议"""
    recommendations = []

    if fail_cnt == 0:
        recommendations.append("✅ **All validations passed.** The portfolio demonstrates consistent pricing and Greeks within expected theoretical bounds.")
    else:
        recommendations.append(f"⚠️ **{fail_cnt} validation failures detected.** Review failed options for potential pricing discrepancies or input data errors.")

    # 检查波动率
    if 'sigma' in vr_df.columns:
        try:
            sigma_numeric = pd.to_numeric(vr_df['sigma'], errors='coerce')
            if sigma_numeric.max() > 1.0:  # 100% volatility
                recommendations.append("⚠️ **High volatility detected** (>100%). Consider reviewing volatility inputs for accuracy.")
        except Exception:
            pass

    # 检查到期时间
    if 'T' in vr_df.columns:
        try:
            T_numeric = pd.to_numeric(vr_df['T'], errors='coerce')
            if T_numeric.min() < 0.1:  # Less than 1.2 months
                recommendations.append("📌 **Short-dated options detected** (T < 0.1 years). Monitor Theta decay closely.")
        except Exception:
            pass

    # 检查 Gamma 风险
    if 'gamma' in vr_df.columns:
        try:
            gamma_numeric = pd.to_numeric(vr_df['gamma'], errors='coerce')
            if gamma_numeric.max() > 0.1:
                recommendations.append("📌 **High Gamma exposure detected.** Portfolio may be sensitive to large moves in underlying asset.")
        except Exception:
            pass

    if not recommendations:
        recommendations.append("✅ No specific recommendations at this time. Continue monitoring market conditions.")

    return "\n".join(recommendations)

def _generate_fallback_summary_md(
    total: int,
    passed: Optional[int],
    failed: Optional[int],
    option_type_counts: Optional[Dict[str, int]],
    key_issues_md: str,
    bsm_df: Optional[pd.DataFrame],
    greeks_df: Optional[pd.DataFrame],
) -> str:
    lines = []
    lines.append(f"# Validation Summary")
    lines.append("")
    lines.append(f"- Date: {datetime.now().strftime('%Y-%m-%d')}")
    lines.append(f"- Total options: {total}")
    lines.append(f"- Passed: {passed if passed is not None else 'N/A'}")
    lines.append(f"- Failed: {failed if failed is not None else 'N/A'}")
    lines.append("")
    lines.append("## Option Types")
    if option_type_counts:
        for k, v in option_type_counts.items():
            lines.append(f"- {k}: {v}")
    else:
        lines.append("- N/A")
    lines.append("")
    lines.append("## Key Issues")
    lines.append(key_issues_md or "- None")
    lines.append("")

    # 计算简要统计
    calc_lines = []
    if bsm_df is not None and bsm_df is not False and "BSM_Price" in bsm_df.columns:
        try:
            bsm_price_numeric = pd.to_numeric(bsm_df['BSM_Price'], errors='coerce')
            calc_lines.append(f"- Avg BSM Price: {bsm_price_numeric.mean():.4f}")
        except Exception:
            pass
    if greeks_df is not None and greeks_df is not False:
        for col in ["delta","gamma","vega","rho","theta"]:
            if col in greeks_df.columns:
                try:
                    col_numeric = pd.to_numeric(greeks_df[col], errors='coerce')
                    calc_lines.append(f"- Avg {col.capitalize()}: {col_numeric.mean():.6f}")
                except Exception:
                    pass
    if calc_lines:
        lines.append("## Calculation Stats")
        lines += calc_lines
        lines.append("")

    lines.append(f"_Generated at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}_")
    return "\n".join(lines)



In [59]:
validate_results = state['validate_results']
bsm_results = state['bsm_results']
greeks_results = state['greeks_results']
vr_df = load_json_as_df(validate_results)
bsm_df = load_json_as_df(bsm_results) if bsm_results is not None else None
greeks_df = load_json_as_df(greeks_results) if greeks_results is not None else None

In [60]:

bsm_df = load_json_as_df(bsm_results) if bsm_results is not None else None
greeks_df = load_json_as_df(greeks_results) if greeks_results is not None else None

total = len(vr_df)
pass_cnt  = int((vr_df.get("validations_result") == "passed").sum()) if "validations_result" in vr_df else None
fail_cnt  = int((vr_df.get("validations_result") == "failed").sum()) if "validations_result" in vr_df else None

option_type_counts = None
if "option_type" in vr_df.columns:
    option_type_counts = vr_df["option_type"].value_counts(dropna=False).to_dict()

key_issues_md = ""
if "validations_details" in vr_df.columns:
    issues = []
    for x in vr_df["validations_details"].tolist():
        if isinstance(x, list):
            issues.extend([str(i) for i in x if i])
    if issues:
        key_issues_md = "\n".join(f"- {i}" for i in issues[:50])  # 防止过长

In [61]:
template_path = str(Path.cwd().parents[1] / "src" / "bsm_multi_agents" / "templates" / "summary_template.md")
template_txt = _load_template_text(template_path)

In [62]:
analysis_date = datetime.now().strftime("%Y-%m-%d")
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# 计算通过率
pass_rate = f"{(pass_cnt / total * 100):.1f}" if total > 0 and pass_cnt is not None else "0.0"
fail_rate = f"{(fail_cnt / total * 100):.1f}" if total > 0 and fail_cnt is not None else "0.0"

# 验证状态
validation_status = "✅ PASSED" if fail_cnt == 0 else f"⚠️ {fail_cnt} ISSUES FOUND"

# === 期权类型分布 ===
option_types_md = ""
if option_type_counts:
    option_types_md = "\n".join(f"  - **{k.capitalize()}:** {v} positions" for k, v in option_type_counts.items())
else:
    option_types_md = "  - N/A"

# === 关键指标 ===
key_metrics = f"- **Average Option Price:** {pd.to_numeric(vr_df.get('price', [0]), errors='coerce').mean():.4f}\n"
key_metrics += f"- **Price Range:** [{pd.to_numeric(vr_df.get('price', [0]), errors='coerce').min():.4f}, {pd.to_numeric(vr_df.get('price', [0]), errors='coerce').max():.4f}]"

# === 标的资产统计 ===
underlying_stats = ""
if 'S' in vr_df.columns:
    S_numeric = pd.to_numeric(vr_df['S'], errors='coerce')
    underlying_stats = f"- **Spot Price Range:** [{S_numeric.min():.2f}, {S_numeric.max():.2f}]\n"
    underlying_stats += f"- **Average Spot:** {S_numeric.mean():.2f}\n"
    underlying_stats += f"- **Volatility (σ) Range:** [{pd.to_numeric(vr_df.get('sigma', [0]), errors='coerce').min():.2%}, {pd.to_numeric(vr_df.get('sigma', [0]), errors='coerce').max():.2%}]"
else:
    underlying_stats = "- Data not available"

# === 行权价分布 ===
strike_distribution = ""
if 'K' in vr_df.columns:
    K_numeric = pd.to_numeric(vr_df['K'], errors='coerce')
    strike_distribution = f"- **Strike Range:** [{K_numeric.min():.2f}, {K_numeric.max():.2f}]\n"
    strike_distribution += f"- **Average Strike:** {K_numeric.mean():.2f}"
else:
    strike_distribution = "- Data not available"

# === 到期时间分布 ===
maturity_profile = ""
if 'T' in vr_df.columns:
    T_numeric = pd.to_numeric(vr_df['T'], errors='coerce')
    maturity_profile = f"- **Time to Maturity Range:** [{T_numeric.min():.2f}, {T_numeric.max():.2f}] years\n"
    maturity_profile += f"- **Average Maturity:** {T_numeric.mean():.2f} years"
else:
    maturity_profile = "- Data not available"

# === BSM 定价摘要 ===
bsm_pricing_summary = "- No BSM pricing data available"
if bsm_df is not None and bsm_df is not False and "BSM_Price" in bsm_df.columns:
    try:
        bsm_price_numeric = pd.to_numeric(bsm_df['BSM_Price'], errors='coerce')
        bsm_pricing_summary = f"- **Total Options Priced:** {len(bsm_df)}\n"
        bsm_pricing_summary += f"- **Average BSM Price:** ${bsm_price_numeric.mean():.4f}\n"
        bsm_pricing_summary += f"- **Price Range:** [${bsm_price_numeric.min():.4f}, ${bsm_price_numeric.max():.4f}]\n"
        bsm_pricing_summary += f"- **Total Portfolio Value:** ${bsm_price_numeric.sum():.2f}"
    except Exception:
        pass


In [63]:
# === Greeks 摘要 ===
greeks_summary = _generate_greeks_summary(vr_df)

# === 验证细节 ===
validation_details = f"**All validations passed criteria:** {pass_cnt}/{total} options"

# === 关键问题 ===
critical_issues = key_issues_md if key_issues_md else "✅ No critical issues identified. All options passed validation checks."

# === Greeks 分析（详细） ===
delta_analysis = _generate_greek_analysis(vr_df, 'delta', 'Delta', '[-1, 1]')
gamma_analysis = _generate_greek_analysis(vr_df, 'gamma', 'Gamma', '[0, ∞)')
vega_analysis = _generate_greek_analysis(vr_df, 'vega', 'Vega', '[0, ∞)')
theta_analysis = _generate_greek_analysis(vr_df, 'theta', 'Theta', '(-∞, 0]')
rho_analysis = _generate_greek_analysis(vr_df, 'rho', 'Rho', 'Varies')

# === 性能摘要 ===
performance_summary = f"Analysis completed successfully for {total} options with {pass_rate}% validation pass rate."

# === 异常值 ===
anomalies = "No significant anomalies detected in the current dataset."

# === 模型准确性 ===
model_accuracy = "BSM model assumptions hold for the analyzed dataset. All Greeks within expected theoretical bounds."

# === 建议 ===
recommendations = _generate_recommendations(fail_cnt, total, vr_df)

# === 分析周期 ===
if 'date' in vr_df.columns:
    dates = pd.to_datetime(vr_df['date'], errors='coerce')
    analysis_period = f"{dates.min().strftime('%Y-%m-%d')} to {dates.max().strftime('%Y-%m-%d')}"
else:
    analysis_period = analysis_date

In [64]:
summary = template_txt.format(
    analysis_date=analysis_date,
    analysis_period=analysis_period,
    total_options=total,
    validation_status=validation_status,
    pass_rate=pass_rate,
    failed_count=fail_cnt if fail_cnt is not None else 0,
    option_types=option_types_md,
    key_metrics=key_metrics,
    underlying_stats=underlying_stats,
    strike_distribution=strike_distribution,
    maturity_profile=maturity_profile,
    bsm_pricing_summary=bsm_pricing_summary,
    greeks_summary=greeks_summary,
    passed_count=pass_cnt if pass_cnt is not None else 0,
    fail_rate=fail_rate,
    validation_details=validation_details,
    critical_issues=critical_issues,
    delta_analysis=delta_analysis,
    gamma_analysis=gamma_analysis,
    vega_analysis=vega_analysis,
    theta_analysis=theta_analysis,
    rho_analysis=rho_analysis,
    performance_summary=performance_summary,
    anomalies=anomalies,
    model_accuracy=model_accuracy,
    recommendations=recommendations,
    timestamp=timestamp,
)

In [65]:
summary

'# OPTIONS PRICING ANALYSIS (OPA)\n## Black-Scholes-Merton Model Validation Report\n\n---\n\n**Report Date:** 2025-11-16\n**Analysis Period:** 2025-09-01 to 2025-09-10\n**Total Instruments Analyzed:** 10\n**Validation Status:** ✅ PASSED\n\n---\n\n## EXECUTIVE SUMMARY\n\n### Validation Overview\n- **Total Options Processed:** 10\n- **Validation Pass Rate:** 100.0%\n- **Failed Validations:** 0\n- **Option Types Distribution:**\n  - **Call:** 5 positions\n  - **Put:** 5 positions\n\n### Key Metrics\n- **Average Option Price:** 5.9433\n- **Price Range:** [2.3917, 8.0214]\n\n---\n\n## MARKET DATA SNAPSHOT\n\n### Underlying Asset Statistics\n- **Spot Price Range:** [96.00, 104.00]\n- **Average Spot:** 100.00\n- **Volatility (σ) Range:** [18.00%, 23.00%]\n\n### Strike Price Distribution\n- **Strike Range:** [101.00, 109.00]\n- **Average Strike:** 105.10\n\n### Time to Maturity Profile\n- **Time to Maturity Range:** [0.10, 1.00] years\n- **Average Maturity:** 0.55 years\n\n---\n\n## PRICING AN

## Check chart_generator

In [5]:
from bsm_multi_agents.agents import data_loader_agent
importlib.reload(data_loader_agent)
from bsm_multi_agents.agents.data_loader_agent import data_loader_node


from bsm_multi_agents.agents import calculator_agent
importlib.reload(calculator_agent)
from bsm_multi_agents.agents.calculator_agent import calculator_node


from bsm_multi_agents.agents import validator_agent
importlib.reload(validator_agent)
from bsm_multi_agents.agents.validator_agent import validator_node

from bsm_multi_agents.agents import summary_generator
importlib.reload(summary_generator)
from bsm_multi_agents.agents.summary_generator import summary_generator_node

In [6]:
csv_path = str(Path.cwd().parents[1] / "data" / "input" / "dummy_options.csv")
state = WorkflowState(csv_file_path=csv_path)
out = data_loader_node(state)
state = {**state, **out}

out = calculator_node(state)
state = {**state, **out}

out = validator_node(state)
state = {**state, **out}

out = summary_generator_node(state)
state = {**state, **out}

[chart_generator tools] ['csv_loader']
[chart_generator tools] ['bsm_calculator', 'batch_bsm_calculator', 'greeks_calculator', 'batch_greeks_calculator', 'sensitivity_test']
[chart_generator tools] ['validate_greeks_rules', 'batch_greeks_validator']
[chart_generator tools] ['generate_summary']


In [7]:
[
    "bsm_results" not in state or not state["bsm_results"],
    "greeks_results" not in state or not state["greeks_results"]
]

[False, False]

In [10]:
from bsm_multi_agents.agents import agent_factory
importlib.reload(agent_factory)
from bsm_multi_agents.agents.agent_factory import built_graph_agent_by_role

In [23]:
from bsm_multi_agents.tools.chart_generator_tools import (
    create_summary_charts,
    create_option_price_chart,
    create_greeks_chart
)


In [24]:
output_dir = str(Path.cwd().parents[1] / "data" / "output")
result_json = create_summary_charts.invoke({
        "bsm_results": json.dumps(state["bsm_results"], ensure_ascii=False),
        "greeks_results": json.dumps(state["greeks_results"], ensure_ascii=False),
        "output_dir": output_dir,
    })

In [26]:
result_jsonmerged_messages = list(state.get("messages", []))
if isinstance(result, dict) and "messages" in result:
    merged_messages.extend(result["messages"])
out = {"messages": merged_messages}

merge_state_update_from_tool_messages(
    result,
    out,
    tool_names=("create_summary_charts",),
)

In [27]:
out

{'messages': [HumanMessage(content='"Load the option data from the CSV file at: /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv"\n\n"Use the csv_loader tool to read the CSV file. Return the data in JSON format."', additional_kwargs={}, response_metadata={}, id='3f1ef52f-d637-4610-8162-c08d34985de4'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:32b', 'created_at': '2025-11-18T00:32:31.212958Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3344434791, 'load_duration': 78772416, 'prompt_eval_count': 267, 'prompt_eval_duration': 1668707375, 'eval_count': 36, 'eval_duration': 1584820459, 'logprobs': None, 'model_name': 'qwen2.5:32b', 'model_provider': 'ollama'}, id='lc_run--a9a24d98-2999-4ac0-abca-7769158694a3-0', tool_calls=[{'name': 'csv_loader', 'args': {'filepath': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv'}, 'id': '8d9345e0-d312-44f7-9396-7c87e3488ec0', 'type': 'tool_call'}], usage_m

In [ ]:
agent_role = "chart_generator"
default_system = """
You are a reporting agent specialized in generating summary charts.
"""
agent = built_graph_agent_by_role(agent_role, default_system=default_system)

output_dir = str(Path.cwd().parents[1] / "data" / "output")

prompt_path = Path.cwd().parents[1] / "src" / "bsm_multi_agents" / "prompts" / "chart_generator_prompts.txt"
user_prompt = load_prompt(prompt_path).format(
    bsm_results=json.dumps(state["bsm_results"], ensure_ascii=False),
    greeks_results=json.dumps(state["greeks_results"], ensure_ascii=False),
    output_dir=output_dir,
)

[chart_generator tools] ['create_option_price_chart', 'create_greeks_chart', 'create_summary_charts']


In [19]:
result = agent.invoke(
        {"messages": [HumanMessage(content=user_prompt)]},
        config={"recursion_limit": 10, "configurable": {"thread_id": state.get("thread_id","run-1")}}
    )

In [20]:
merged_messages = list(state.get("messages", []))
if isinstance(result, dict) and "messages" in result:
    merged_messages.extend(result["messages"])
out = {"messages": merged_messages}

In [21]:
merge_state_update_from_tool_messages(
    result,
    out,
    tool_names=("create_summary_charts",),
)

In [22]:
print_resp(out)

Step 1 - inputs:
   "Load the option data from the CSV file at: /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv"

"Use the csv_loader tool to read the CSV file. Return the data in JSON format."

Step 2 - Agent decide tools used:
   Tool name: csv_loader
   Tool parameters: {'filepath': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv'}

Step 3 - outputs:
   Tool name: csv_loader
   Outputs: {"state_update": {"csv_data": [{"date": "2025-09-01", "S": 100, "K": 105, "T": 1.0, "r": 0.05, "sigma": 0.2, "option_type": "call"}, {"date": "2025-09-02", "S": 102, "K": 106, "T": 0.9, "r": 0.045, "sigma": 0.19, "option_type": "put"}, {"date": "2025-09-03", "S": 98, "K": 104, "T": 0.8, "r": 0.048, "sigma": 0.21, "option_type": "call"}, {"date": "2025-09-04", "S": 101, "K": 107, "T": 0.7, "r": 0.047, "sigma": 0.18, "option_type": "call"}, {"date": "2025-09-05", "S": 99, "K": 103, "T": 0.6, "r": 0.046, "sigma": 0.22, "option_type": "put"}, {"date": "202

In [16]:
out

{'messages': [HumanMessage(content='"Load the option data from the CSV file at: /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv"\n\n"Use the csv_loader tool to read the CSV file. Return the data in JSON format."', additional_kwargs={}, response_metadata={}, id='3f1ef52f-d637-4610-8162-c08d34985de4'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:32b', 'created_at': '2025-11-18T00:32:31.212958Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3344434791, 'load_duration': 78772416, 'prompt_eval_count': 267, 'prompt_eval_duration': 1668707375, 'eval_count': 36, 'eval_duration': 1584820459, 'logprobs': None, 'model_name': 'qwen2.5:32b', 'model_provider': 'ollama'}, id='lc_run--a9a24d98-2999-4ac0-abca-7769158694a3-0', tool_calls=[{'name': 'csv_loader', 'args': {'filepath': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv'}, 'id': '8d9345e0-d312-44f7-9396-7c87e3488ec0', 'type': 'tool_call'}], usage_m

In [17]:
# ========================================
# 诊断代码：测试工具直接调用
# ========================================
import json
from pathlib import Path

print("=" * 80)
print("测试1: 直接调用 create_summary_charts 工具")
print("=" * 80)

from bsm_multi_agents.tools.chart_generator_tools import create_summary_charts

# 准备测试数据
test_bsm = state["bsm_results"][:2]  # 只取前2条数据测试
test_greeks = state["greeks_results"][:2]

output_dir = str(Path.cwd().parents[1] / "data" / "output")
bsm_json = json.dumps(test_bsm, ensure_ascii=False)
greeks_json = json.dumps(test_greeks, ensure_ascii=False)

print(f"输出目录: {output_dir}")
print(f"bsm_json 类型: {type(bsm_json)}")
print(f"bsm_json 前150字符: {bsm_json[:150]}")

# 直接调用工具
try:
    result_str = create_summary_charts.invoke({
        "bsm_results": bsm_json,
        "greeks_results": greeks_json,
        "output_dir": output_dir
    })
    print("\n✅ 工具直接调用成功!")
    result_dict = json.loads(result_str)
    print(f"返回结果: {json.dumps(result_dict, indent=2, ensure_ascii=False)}")
    
    # 检查文件是否生成
    if result_dict.get("status") == "success":
        for chart in result_dict.get("charts", []):
            chart_path = chart.get("chart_path")
            if chart_path and Path(chart_path).exists():
                print(f"✅ 图片已生成: {chart_path}")
            else:
                print(f"❌ 图片未找到: {chart_path}")
except Exception as e:
    print(f"\n❌ 工具调用失败: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "=" * 80)
print("测试2: 检查 Agent 收到的提示词")
print("=" * 80)

# 检查提示词内容
prompt_path = Path.cwd().parents[1] / "src" / "bsm_multi_agents" / "prompts" / "chart_generator_prompts.txt"
user_prompt = load_prompt(prompt_path).format(
    bsm_results=bsm_json,
    greeks_results=greeks_json,
    output_dir=output_dir,
)

print(f"提示词长度: {len(user_prompt)} 字符")
print(f"提示词内容:\n{user_prompt}\n")

print("=" * 80)
print("测试3: 检查工具是否正确注册")
print("=" * 80)

tools = get_tools_for_role("chart_generator")
print(f"找到 {len(tools)} 个工具:")
for tool in tools:
    print(f"  - {tool.name}")
    print(f"    参数: {tool.args}")

测试1: 直接调用 create_summary_charts 工具
输出目录: /Users/yifanli/Github/model_doc_automation/data/output
bsm_json 类型: <class 'str'>
bsm_json 前150字符: [{"K": 105, "S": 100, "T": 1.0, "date": "2025-09-01", "option_type": "call", "r": 0.05, "sigma": 0.2, "BSM_Price": "8.021352235143176"}, {"K": 106, "S

✅ 工具直接调用成功!
返回结果: {
  "status": "success",
  "charts": [
    {
      "status": "success",
      "chart_path": "/Users/yifanli/Github/model_doc_automation/data/output/option_prices.png",
      "description": "Option prices visualization showing relationship between prices and spot/time"
    },
    {
      "status": "success",
      "chart_path": "/Users/yifanli/Github/model_doc_automation/data/output/greeks_sensitivity.png",
      "description": "Greeks sensitivity analysis showing how each Greek changes with spot price"
    }
  ],
  "total_charts": 2
}
✅ 图片已生成: /Users/yifanli/Github/model_doc_automation/data/output/option_prices.png
✅ 图片已生成: /Users/yifanli/Github/model_doc_automation/data/output/gre

### Check create_option_price_chart tool

In [16]:
import matplotlib
matplotlib.use("MacOSX")
import matplotlib.pyplot as plt

In [17]:
from bsm_multi_agents.agents import data_loader_agent
importlib.reload(data_loader_agent)
from bsm_multi_agents.agents.data_loader_agent import data_loader_node


from bsm_multi_agents.agents import calculator_agent
importlib.reload(calculator_agent)
from bsm_multi_agents.agents.calculator_agent import calculator_node


from bsm_multi_agents.agents import validator_agent
importlib.reload(validator_agent)
from bsm_multi_agents.agents.validator_agent import validator_node

from bsm_multi_agents.agents import summary_generator
importlib.reload(summary_generator)
from bsm_multi_agents.agents.summary_generator import summary_generator_node

In [7]:
csv_path = str(Path.cwd().parents[1] / "data" / "input" / "dummy_options.csv")
state = WorkflowState(csv_file_path=csv_path)
out = data_loader_node(state)
state = {**state, **out}

out = calculator_node(state)
state = {**state, **out}

out = validator_node(state)
state = {**state, **out}

out = summary_generator_node(state)
state = {**state, **out}

In [18]:
state.keys()

dict_keys(['csv_file_path', 'messages', 'csv_data', 'greeks_results', 'bsm_results', 'validate_results', 'report_md', 'report_path'])

In [ ]:
bsm_results = json.dumps(state["bsm_results"], ensure_ascii=False)
output_dir = str(Path.cwd().parents[1] / "data" / "output")

In [21]:
data = json.loads(bsm_results)
# Create output directory
output_path = Path(output_dir)
output_path.mkdir(parents=True, exist_ok=True)
df = pd.DataFrame(data)
df

,K,S,T,date,option_type,r,sigma,BSM_Price
0,105,100,1.0,2025-09-01,call,0.050,0.20,8.021352235143176
1,106,102,0.9,2025-09-02,put,0.045,0.19,7.214238371309641
2,104,98,0.8,2025-09-03,call,0.048,0.21,6.415749358967169
3,107,101,0.7,2025-09-04,call,0.047,0.18,4.952961817553408
4,103,99,0.6,2025-09-05,put,0.046,0.22,7.377696042475904
5,108,103,0.5,2025-09-06,put,0.049,0.20,7.143363876009964
6,102,97,0.4,2025-09-07,call,0.044,0.23,4.250515622426249
7,106,100,0.3,2025-09-08,call,0.050,0.19,2.3916626013541915
8,109,104,0.2,2025-09-09,put,0.045,0.21,6.301796350448868
9,101,96,0.1,2025-09-10,put,0.048,0.20,5.364072939656609


In [22]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

In [23]:
'BSM_Price' in df.columns and 'S' in df.columns

True

In [24]:
calls = df[df['option_type'].str.lower() == 'call']
puts = df[df['option_type'].str.lower() == 'put']

if not calls.empty:
    ax1.plot(calls['S'], calls['BSM_Price'], 'o-', label='Call', color='green', linewidth=2)
if not puts.empty:
    ax1.plot(puts['S'], puts['BSM_Price'], 's-', label='Put', color='red', linewidth=2)

ax1.set_xlabel('Spot Price (S)', fontsize=12)
ax1.set_ylabel('Option Price', fontsize=12)
ax1.set_title('BSM Option Prices vs Spot Price', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

In [25]:
'BSM_Price' in df.columns and 'T' in df.columns

True

In [26]:
calls = df[df['option_type'].str.lower() == 'call']
puts = df[df['option_type'].str.lower() == 'put']

if not calls.empty:
    ax2.plot(calls['T'], calls['BSM_Price'], 'o-', label='Call', color='green', linewidth=2)
if not puts.empty:
    ax2.plot(puts['T'], puts['BSM_Price'], 's-', label='Put', color='red', linewidth=2)

ax2.set_xlabel('Time to Maturity (years)', fontsize=12)
ax2.set_ylabel('Option Price', fontsize=12)
ax2.set_title('BSM Option Prices vs Time to Maturity', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

In [27]:
plt.tight_layout()

In [28]:
plt.show()

In [29]:
# Save chart
chart_path = output_path / "option_prices.png"
plt.savefig(chart_path, dpi=300, bbox_inches='tight')
plt.close()

### Check create_greeks_chart tool

In [30]:
greeks_results = json.dumps(state["greeks_results"], ensure_ascii=False)
output_dir = str(Path.cwd().parents[1] / "data" / "output")

In [31]:
data = json.loads(greeks_results)

# Create output directory
output_path = Path(output_dir)
output_path.mkdir(parents=True, exist_ok=True)

# For sensitivity test data (list of dicts)
df = pd.DataFrame(data)
df

,K,S,T,date,option_type,r,sigma,price,delta,gamma,vega,rho,theta
0,105,100,1.0,2025-09-01,call,0.050,0.20,8.021352,0.542228,0.019835,39.670524,46.201481,-6.277126
1,106,102,0.9,2025-09-02,put,0.045,0.19,7.214238,-0.459613,0.021587,38.405945,-48.685326,-1.619695
2,104,98,0.8,2025-09-03,call,0.048,0.21,6.415749,0.492814,0.021670,34.963160,33.504030,-6.599157
3,107,101,0.7,2025-09-04,call,0.047,0.18,4.952962,0.464369,0.026123,33.577140,29.363842,-6.288633
4,103,99,0.6,2025-09-05,put,0.046,0.22,7.377696,-0.494122,0.023644,30.589596,-33.777473,-3.018486
5,108,103,0.5,2025-09-06,put,0.049,0.20,7.143364,-0.536346,0.027274,28.935094,-31.193507,-2.730055
6,102,97,0.4,2025-09-07,call,0.044,0.23,4.250516,0.439672,0.027950,24.194012,15.359051,-8.645274
7,106,100,0.3,2025-09-08,call,0.050,0.19,2.391663,0.358024,0.035881,20.452195,10.023233,-8.147067
8,109,104,0.2,2025-09-09,put,0.045,0.21,6.301796,-0.639531,0.038321,17.408093,-14.562611,-5.862661
9,101,96,0.1,2025-09-10,put,0.048,0.20,5.364073,-0.756555,0.051599,9.510738,-7.799334,-5.767057


In [32]:
# Ensure numeric columns
numeric_cols = ['S', 'K', 'delta', 'gamma', 'vega', 'theta', 'rho', 'price', 'T']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Calculate Moneyness
if 'S' in df.columns and 'K' in df.columns and 'option_type' in df.columns:
    df['moneyness_ratio'] = df['S'] / df['K']

    def classify_moneyness(row):
        ratio = row['moneyness_ratio']
        opt_type = str(row['option_type']).lower()

        if 0.95 <= ratio <= 1.05:
            return 'ATM'
        elif opt_type == 'call':
            return 'ITM' if ratio > 1.05 else 'OTM'
        else:  # put
            return 'ITM' if ratio < 0.95 else 'OTM'

    df['moneyness'] = df.apply(classify_moneyness, axis=1)

In [33]:
# Create figure with subplots
fig = plt.figure(figsize=(18, 12))

# Define colors
call_color = '#2E86AB'  # Blue for calls
put_color = '#A23B72'   # Red/Magenta for puts

# ===== Subplot 1: Portfolio Greeks Summary Table =====
ax1 = plt.subplot(2, 3, 1)
ax1.axis('tight')
ax1.axis('off')

# Calculate portfolio Greeks
greeks_cols = ['delta', 'gamma', 'vega', 'theta', 'rho']
summary_data = []

for greek in greeks_cols:
    if greek in df.columns:
        total = df[greek].sum()
        call_val = df[df['option_type'].str.lower() == 'call'][greek].sum() if 'option_type' in df.columns else 0
        put_val = df[df['option_type'].str.lower() == 'put'][greek].sum() if 'option_type' in df.columns else 0
        summary_data.append([greek.capitalize(), f'{total:.4f}', f'{call_val:.4f}', f'{put_val:.4f}'])

table = ax1.table(cellText=summary_data,
                    colLabels=['Greek', 'Portfolio', 'Calls', 'Puts'],
                    cellLoc='center',
                    loc='center',
                    colWidths=[0.2, 0.25, 0.25, 0.25])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)

# Style header
for i in range(4):
    table[(0, i)].set_facecolor('#4472C4')
    table[(0, i)].set_text_props(weight='bold', color='white')

ax1.set_title('Portfolio Greeks Summary', fontsize=14, fontweight='bold', pad=20)

Text(0.5, 1.0, 'Portfolio Greeks Summary')

In [34]:
# ===== Subplot 2: Delta Distribution =====
ax2 = plt.subplot(2, 3, 2)
if 'delta' in df.columns and 'option_type' in df.columns:
    calls = df[df['option_type'].str.lower() == 'call']['delta']
    puts = df[df['option_type'].str.lower() == 'put']['delta']

    positions = np.arange(len(df))
    width = 0.35

    call_positions = positions[df['option_type'].str.lower() == 'call']
    put_positions = positions[df['option_type'].str.lower() == 'put']

    if len(call_positions) > 0:
        ax2.bar(call_positions, calls, width, label='Call', color=call_color, alpha=0.8)
    if len(put_positions) > 0:
        ax2.bar(put_positions, puts, width, label='Put', color=put_color, alpha=0.8)

    ax2.set_xlabel('Option Index', fontsize=10)
    ax2.set_ylabel('Delta (Δ)', fontsize=10)
    ax2.set_title('Delta Distribution by Option Type', fontsize=12, fontweight='bold')
    ax2.axhline(y=0, color='k', linestyle='--', alpha=0.3)
    ax2.legend()
    ax2.grid(True, alpha=0.3, axis='y')


In [35]:
# ===== Subplot 3: Gamma Distribution =====
ax3 = plt.subplot(2, 3, 3)
if 'gamma' in df.columns and 'option_type' in df.columns:
    calls_gamma = df[df['option_type'].str.lower() == 'call']['gamma']
    puts_gamma = df[df['option_type'].str.lower() == 'put']['gamma']

    call_positions = positions[df['option_type'].str.lower() == 'call']
    put_positions = positions[df['option_type'].str.lower() == 'put']

    if len(call_positions) > 0:
        ax3.bar(call_positions, calls_gamma, width, label='Call', color=call_color, alpha=0.8)
    if len(put_positions) > 0:
        ax3.bar(put_positions, puts_gamma, width, label='Put', color=put_color, alpha=0.8)

    ax3.set_xlabel('Option Index', fontsize=10)
    ax3.set_ylabel('Gamma (Γ)', fontsize=10)
    ax3.set_title('Gamma Distribution', fontsize=12, fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3, axis='y')

In [36]:
# ===== Subplot 4: Vega Distribution =====
ax4 = plt.subplot(2, 3, 4)
if 'vega' in df.columns and 'option_type' in df.columns:
    calls_vega = df[df['option_type'].str.lower() == 'call']['vega']
    puts_vega = df[df['option_type'].str.lower() == 'put']['vega']

    call_positions = positions[df['option_type'].str.lower() == 'call']
    put_positions = positions[df['option_type'].str.lower() == 'put']

    if len(call_positions) > 0:
        ax4.bar(call_positions, calls_vega, width, label='Call', color=call_color, alpha=0.8)
    if len(put_positions) > 0:
        ax4.bar(put_positions, puts_vega, width, label='Put', color=put_color, alpha=0.8)

    ax4.set_xlabel('Option Index', fontsize=10)
    ax4.set_ylabel('Vega (ν)', fontsize=10)
    ax4.set_title('Vega Distribution', fontsize=12, fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3, axis='y')

In [37]:
# ===== Subplot 5: Greeks by Moneyness =====
ax5 = plt.subplot(2, 3, 5)
if 'moneyness' in df.columns and all(col in df.columns for col in ['delta', 'gamma', 'vega']):
    moneyness_groups = df.groupby('moneyness')[['delta', 'gamma', 'vega']].mean()

    x_pos = np.arange(len(moneyness_groups))
    width = 0.25

    if 'delta' in moneyness_groups.columns:
        ax5.bar(x_pos - width, moneyness_groups['delta'], width, label='Avg Delta', color='#2E86AB')
    if 'gamma' in moneyness_groups.columns:
        ax5.bar(x_pos, moneyness_groups['gamma'] * 10, width, label='Avg Gamma (×10)', color='#F18F01')
    if 'vega' in moneyness_groups.columns:
        ax5.bar(x_pos + width, moneyness_groups['vega'] / 10, width, label='Avg Vega (÷10)', color='#C73E1D')

    ax5.set_xlabel('Moneyness', fontsize=10)
    ax5.set_ylabel('Greek Value', fontsize=10)
    ax5.set_title('Greeks by Moneyness', fontsize=12, fontweight='bold')
    ax5.set_xticks(x_pos)
    ax5.set_xticklabels(moneyness_groups.index)
    ax5.legend()
    ax5.grid(True, alpha=0.3, axis='y')

In [38]:
# ===== Subplot 6: Theta & Rho Summary =====
ax6 = plt.subplot(2, 3, 6)
if 'theta' in df.columns and 'rho' in df.columns:
    positions = np.arange(len(df))

    ax6_twin = ax6.twinx()

    ax6.bar(positions - width/2, df['theta'], width, label='Theta (Θ)', color='#6A4C93', alpha=0.7)
    ax6_twin.bar(positions + width/2, df['rho'], width, label='Rho (ρ)', color='#1982C4', alpha=0.7)

    ax6.set_xlabel('Option Index', fontsize=10)
    ax6.set_ylabel('Theta (Θ)', fontsize=10, color='#6A4C93')
    ax6_twin.set_ylabel('Rho (ρ)', fontsize=10, color='#1982C4')
    ax6.set_title('Theta & Rho Summary', fontsize=12, fontweight='bold')

    ax6.tick_params(axis='y', labelcolor='#6A4C93')
    ax6_twin.tick_params(axis='y', labelcolor='#1982C4')

    # Combined legend
    lines1, labels1 = ax6.get_legend_handles_labels()
    lines2, labels2 = ax6_twin.get_legend_handles_labels()
    ax6.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

    ax6.grid(True, alpha=0.3, axis='y')
plt.tight_layout()

In [39]:
plt.show()

In [40]:
chart_path = output_path / "greeks_sensitivity.png"
plt.savefig(chart_path, dpi=300, bbox_inches='tight')
plt.close()

### Check create_summary_charts tool

In [41]:
from bsm_multi_agents.tools import chart_generator_tools
importlib.reload(chart_generator_tools)
from bsm_multi_agents.tools.chart_generator_tools import create_option_price_chart,create_greeks_chart

In [42]:
bsm_results = json.dumps(state["bsm_results"], ensure_ascii=False)
greeks_results = json.dumps(state["greeks_results"], ensure_ascii=False)

In [ ]:
output_dir = str(Path.cwd().parents[1] / "data" / "output")
bsm_results = json.dumps(bsm_results)
greeks_results = json.dumps(greeks_results)

charts = []

In [44]:
price_result = create_option_price_chart.invoke({
    "bsm_results": bsm_results,
    "output_dir": output_dir
})
price_info = json.loads(price_result)
if price_info.get("status") == "success":
    charts.append(price_info)

# Create Greeks chart
greeks_result = create_greeks_chart.invoke({
    "greeks_results": greeks_results,
    "output_dir": output_dir
})
greeks_info = json.loads(greeks_result)
if greeks_info.get("status") == "success":
    charts.append(greeks_info)


In [47]:
price_info

{'status': 'success',
 'chart_path': '/Users/yifanli/Github/model_doc_automation/data/output/option_prices.png',
 'description': 'Option prices visualization showing relationship between prices and spot/time'}

In [46]:
json.dumps({
            "status": "success",
            "charts": charts,
            "total_charts": len(charts)
        })

'{"status": "success", "charts": [{"status": "success", "chart_path": "/Users/yifanli/Github/model_doc_automation/data/output/option_prices.png", "description": "Option prices visualization showing relationship between prices and spot/time"}, {"status": "success", "chart_path": "/Users/yifanli/Github/model_doc_automation/data/output/greeks_sensitivity.png", "description": "Greeks sensitivity analysis showing how each Greek changes with spot price"}], "total_charts": 2}'

## Check report_generator_node

In [7]:
from bsm_multi_agents.agents import data_loader_agent
importlib.reload(data_loader_agent)
from bsm_multi_agents.agents.data_loader_agent import data_loader_node


from bsm_multi_agents.agents import calculator_agent
importlib.reload(calculator_agent)
from bsm_multi_agents.agents.calculator_agent import calculator_node


from bsm_multi_agents.agents import validator_agent
importlib.reload(validator_agent)
from bsm_multi_agents.agents.validator_agent import validator_node

from bsm_multi_agents.agents import summary_generator_agent
importlib.reload(summary_generator_agent)
from bsm_multi_agents.agents.summary_generator_agent import summary_generator_node

from bsm_multi_agents.agents import chart_generator_agent
importlib.reload(chart_generator_agent)
from bsm_multi_agents.agents.chart_generator_agent import chart_generator_node

from bsm_multi_agents.agents import report_generator_agent
importlib.reload(report_generator_agent)
from bsm_multi_agents.agents.report_generator_agent import report_generator_node

In [8]:
csv_path = str(Path.cwd().parents[1] / "data" / "input" / "dummy_options.csv")
state = WorkflowState(csv_file_path=csv_path)
out = data_loader_node(state)
state = {**state, **out}

out = calculator_node(state)
state = {**state, **out}

out = validator_node(state)
state = {**state, **out}

out = summary_generator_node(state)
state = {**state, **out}

out = chart_generator_node(state)
state = {**state, **out}

out = report_generator_node(state)
state = {**state, **out}

[chart_generator tools] ['csv_loader']
[chart_generator tools] ['bsm_calculator', 'batch_bsm_calculator', 'greeks_calculator', 'batch_greeks_calculator', 'sensitivity_test']
[chart_generator tools] ['validate_greeks_rules', 'batch_greeks_validator']


In [9]:
state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'messages': [HumanMessage(content='"Load the option data from the CSV file at: /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv"\n\n"Use the csv_loader tool to read the CSV file. Return the data in JSON format."', additional_kwargs={}, response_metadata={}, id='bd88757b-213b-4e93-9ce6-2234435917d1'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:32b', 'created_at': '2025-11-18T01:34:10.256453Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6367513500, 'load_duration': 3269522708, 'prompt_eval_count': 267, 'prompt_eval_duration': 1510616625, 'eval_count': 36, 'eval_duration': 1573703542, 'model_name': 'qwen2.5:32b', 'model_provider': 'ollama'}, id='lc_run--604b9515-71cc-4742-9747-a98632e8acdb-0', tool_calls=[{'name': 'csv_loader', 'args': {'filepath': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv'},

In [8]:
markdown_path = None
if ("report_md" in state and state["report_md"]):
    print(1)

1


In [9]:
report_md = state["report_md"]
isinstance(report_md, list) and len(report_md) > 0

True

In [10]:
isinstance(report_md[0], dict)

True

In [11]:
markdown_path = report_md[0].get("file_path") or report_md[0].get("path")
markdown_path

'/Users/yifanli/Github/model_doc_automation/data/output/validation_summary_20251117_202832.md'

In [12]:
chart_paths = []
if "chart_results" in state and state["chart_results"]:
    chart_results = state["chart_results"]
    if isinstance(chart_results, list):
        for chart in chart_results:
            if isinstance(chart, dict):
                path = chart.get("chart_path")
                if path:
                    chart_paths.append(path)
            elif isinstance(chart, str):
                chart_paths.append(chart)
chart_paths

['/Users/yifanli/Github/model_doc_automation/data/output/option_prices.png',
 '/Users/yifanli/Github/model_doc_automation/data/output/greeks_sensitivity.png']

In [13]:
output_dir = str(Path.cwd().parents[1] / "data" / "output")
timestamp = Path(markdown_path).stem.split('_')[-1] if '_' in Path(markdown_path).stem else "latest"
output_path = f"{output_dir}/OPA_Report_{timestamp}.docx"
output_path

'/Users/yifanli/Github/model_doc_automation/data/output/OPA_Report_202832.docx'

In [14]:
from bsm_multi_agents.tools.report_generator_tools import create_word_report

In [15]:
result_json = create_word_report.invoke({
    "markdown_path": markdown_path,
    "chart_paths": json.dumps(chart_paths) if chart_paths else "[]",
    "output_path": output_path,
    "title": "Option Portfolio Analysis Report"
})

In [16]:
result_json

'{"status": "success", "document_path": "/Users/yifanli/Github/model_doc_automation/data/output/OPA_Report_202832.docx", "message": "Word report generated successfully"}'

In [8]:
from bsm_multi_agents.agents import report_generator_agent
importlib.reload(report_generator_agent)
from bsm_multi_agents.agents.report_generator_agent import report_generator_node

In [10]:
out = report_generator_node(state)
state = {**state, **out}

## Check Graph

In [11]:
from bsm_multi_agents.graph import agent_graph
importlib.reload(agent_graph)
from bsm_multi_agents.graph.agent_graph import build_app, WorkflowState

In [12]:
app = build_app()
csv_path = Path.cwd().parents[1] / "data" / "input" / "dummy_options.csv"
init_state: WorkflowState = {
    "csv_file_path": str(csv_path),
    "messages": [HumanMessage(content=f"Load CSV from: {csv_path}")],
}
final_state = app.invoke(
    init_state,
    config={"configurable": {"thread_id": "run-1"}}
)

[chart_generator tools] ['csv_loader']
[chart_generator tools] ['bsm_calculator', 'batch_bsm_calculator', 'greeks_calculator', 'batch_greeks_calculator', 'sensitivity_test']
[chart_generator tools] ['validate_greeks_rules', 'batch_greeks_validator']


In [13]:
print("="*80)
print("验证 Graph 执行结果")
print("="*80)

# 1. 检查必需字段
checks = {
    "csv_data": "✅ CSV 数据已加载",
    "bsm_results": "✅ BSM 价格计算完成",
    "greeks_results": "✅ Greeks 计算完成",
    "validate_results": "✅ Validation 计算完成",
}

all_passed = True

验证 Graph 执行结果


In [14]:
for field, success_msg in checks.items():
    if field in final_state and final_state[field]:
        data = final_state[field]
        count = len(data) if isinstance(data, list) else 1
        print(f"{success_msg} ({count} 条记录)")
    else:
        print(f"❌ {field} 缺失或为空")
        all_passed = False

✅ CSV 数据已加载 (10 条记录)
✅ BSM 价格计算完成 (10 条记录)
✅ Greeks 计算完成 (10 条记录)
✅ Validation 计算完成 (10 条记录)


In [15]:
if "errors" in final_state and final_state["errors"]:
    print(f"\n❌ 发现错误:")
    for error in final_state["errors"]:
        print(f"   {error}")
    all_passed = False

In [16]:
if all_passed:
    print("\n" + "="*80)
    print("数据样例")
    print("="*80)

    # BSM 结果
    if "bsm_results" in final_state and final_state["bsm_results"]:
        bsm = final_state["bsm_results"][0]
        print(f"\nBSM 结果 (第1条):")
        print(f"  期权类型: {bsm.get('option_type')}")
        print(f"  标的价格 S: {bsm.get('S')}")
        print(f"  行权价 K: {bsm.get('K')}")
        print(f"  BSM 价格: {bsm.get('BSM_Price', 'N/A')}")

    # Greeks 结果
    if "greeks_results" in final_state and final_state["greeks_results"]:
        greeks = final_state["greeks_results"][0]
        print(f"\nGreeks 结果 (第1条):")
        print(f"  Delta: {greeks.get('delta')}")
        print(f"  Gamma: {greeks.get('gamma')}")
        print(f"  Vega: {greeks.get('vega')}")
        print(f"  Theta: {greeks.get('theta')}")
        print(f"  Rho: {greeks.get('rho')}")

    if "validate_results" in final_state and final_state["validate_results"]:
        validation = final_state["validate_results"][0]
        print(f"\nValidation 结果 (第1条):")
        print(f"  validations_result: {validation.get('validations_result')}")
        print(f"  validations_details: {validation.get('validations_details')}")


数据样例

BSM 结果 (第1条):
  期权类型: call
  标的价格 S: 100
  行权价 K: 105
  BSM 价格: 8.021352235143176

Greeks 结果 (第1条):
  Delta: 0.5422283336
  Gamma: 0.0198352619
  Vega: 39.6705238084
  Theta: -6.277126437
  Rho: 46.2014811233

Validation 结果 (第1条):
  validations_result: passed
  validations_details: []


In [17]:
print("\n" + "="*80)
if all_passed:
    print("🎉 验证通过！Graph 成功运行！")
else:
    print("❌ 验证失败，请检查上述错误")
print("="*80)




🎉 验证通过！Graph 成功运行！
